In [10]:
# dependencies

import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import datetime
import plotly.express as px
import psycopg2

from sqlalchemy import create_engine

pd.set_option('display.max_columns', None)
pd.options.display.float_format = '{:20,.2f}'.format

In [2]:
# database connection

db_password = "UnitCircle42!"
db_user = "postgres"
db_name = "dot"
endpoint = "awakedb.cre3f7yk1unp.us-west-1.rds.amazonaws.com"

connection_string = f"postgresql://{db_user}:{db_password}@{endpoint}:5432/{db_name}"
engine = create_engine(connection_string)

In [3]:
dot = pd.read_sql('SELECT * FROM invoice_clean;', con = engine)
unl = pd.read_sql('SELECT * FROM unleashed_clean;', con = engine)

unl.columns = ['month','year','customer','date','item','qty','cad','usd','sale_origin','market_segment','parent_customer']
dot.columns = ['month','year','customer','date','item','qty','usd','cad','sale_origin','parent_customer','market_segment']

In [4]:
# fix columns in dot table to match unl table

new_order = ['month', 'year', 'date', 'customer', 'item',
       'qty', 'cad', 'usd', 'sale_origin', 'market_segment',
       'parent_customer']

for column in new_order:
    dot[column] = dot.pop(column)

In [5]:
## concat datasets

lvl2 = pd.concat([dot,unl]).sort_values(by='date',ascending=False).reset_index(drop=True)
lvl2 = lvl2[lvl2.customer != 'DOT Foods, Inc.']
lvl2 = lvl2.convert_dtypes()
lvl2.sample(3)

,month,year,date,customer,item,qty,cad,usd,sale_origin,market_segment,parent_customer
143647,January,2021,2021-01-15,Shopify Customer - AWAKE Energy USA,30ct - AWAKE Chocolate Bites USA -Variety,1.0,26.81,20.1075,unl,Online Direct,Shopify
204174,October,2019,2019-10-11,Amazon FBM,Milk Chocolate Bars 12 Pack,1.0,15.51,11.6325,unl,Online Distributor,Amazon
181873,June,2020,2020-06-16,Shopify Customer - AWAKE Energy USA,50ct Change Maker - AWAKE Chocolate Bites USA ...,1.0,39.79,29.8425,unl,Online Direct,Shopify


In [6]:
new_order = ['date', 'sale_origin', 'market_segment', 'parent_customer', 'customer', 'item', 'qty', 'usd', 'cad','month','year']

for i,col in enumerate(new_order):
    tmp = lvl2[col]
    lvl2.drop(labels=[col],axis=1,inplace=True)
    lvl2.insert(i,col,tmp) 

lvl2.head(2)

,date,sale_origin,market_segment,parent_customer,customer,item,qty,usd,cad,month,year
0,2024-01-18,unl,Online Distributor,Amazon,Amazon SC FBA,50ct Pouch - Variety Pk Choc Bites,1.0,34.9875,46.65,January,2024
1,2024-01-18,unl,Online Distributor,Amazon,Amazon SC FBA,50ct Pouch - PB Milk Choc Bites,1.0,34.9875,46.65,January,2024


In [7]:
##### CAREFUL!!!! REPLACES ENTIRE CLEAN LEVEL_2 DATABASE

lvl2.to_sql('level_2', engine, if_exists='replace', index=False)

243

In [11]:
##  ------ READ ALL DATA INTO APPLICATION

def get_data_from_csv():
    df = pd.read_sql("""
            SELECT * 
            FROM level_2
            WHERE year > '2020'
            """
            ,con = engine)
    return df
df = get_data_from_csv()

### MASTER DATA ###
all_sales = df.copy()
all_sales

,date,sale_origin,market_segment,parent_customer,customer,item,qty,usd,cad,month,year
0,2024-01-18,unl,Online Distributor,Amazon,Amazon SC FBA,50ct Pouch - Variety Pk Choc Bites,1.00,34.99,46.65,January,2024
1,2024-01-18,unl,Online Distributor,Amazon,Amazon SC FBA,50ct Pouch - PB Milk Choc Bites,1.00,34.99,46.65,January,2024
2,2024-01-18,unl,Online Distributor,Amazon,Amazon SC FBA,50ct Pouch - PB Milk Choc Bites,1.00,34.99,46.65,January,2024
3,2024-01-18,unl,Online Distributor,Amazon,Amazon SC FBA,50ct Pouch - Mint Choc Bites,1.00,34.99,46.65,January,2024
4,2024-01-18,unl,Online Distributor,Amazon,Amazon SC FBA,50ct Pouch - Caramel Milk Choc Bites,1.00,34.99,46.65,January,2024
...,...,...,...,...,...,...,...,...,...,...,...
144432,2021-01-04,dot,Vending,Vistar,VISTAR-HOUSTON,CAFFEINATED CHOCOLATE BITES SINGLES,2.00,240.00,319.20,January,2021
144433,2021-01-04,unl,Online Direct,Shopify,Shopify Customer - AWAKE Energy USA,4 Free Bites,1.00,0.00,0.00,January,2021
144434,2021-01-04,unl,Online Direct,Shopify,Shopify Customer - AWAKE Energy USA,4 Free Bites,1.00,0.00,0.00,January,2021
144435,2021-01-04,unl,Online Direct,Shopify,Shopify Customer - AWAKE Energy USA,Shopify Shipping cost,1.00,3.98,5.31,January,2021


In [12]:
all_sales.groupby('year')['usd'].sum()

year
2021           5,139,913.89
2022           7,467,696.66
2023           9,614,740.63
2024             306,266.59
Name: usd, dtype: float64